In [1]:
## INITIALIZE ##
from dotenv import load_dotenv
import os
from langgraph_app import app_graph
from IPython.display import Markdown, display, HTML
import pprint

# Load environment variables from .env file in the current directory
load_dotenv()

# Enable automatic reloading
%load_ext autoreload
%autoreload 2

In [2]:
## SELECT FILES ##
import os
import glob
import warnings

# Suppress tkinter warnings on macOS
warnings.filterwarnings("ignore", category=DeprecationWarning)

try:
    import tkinter as tk
    from tkinter import filedialog
    
    # Suppress macOS console warnings
    import sys
    from contextlib import redirect_stderr
    from io import StringIO
    
    # Create a root window and hide it
    root = tk.Tk()
    root.withdraw()  # Hide the main window
    root.attributes('-topmost', True)  # Bring dialog to front
    
    # Set the initial directory to the images folder
    initial_dir = os.path.abspath("images")
    
    print("Opening file selection dialog...")
    print("You can select multiple files by holding Cmd (Mac) or Ctrl (Windows/Linux)")
    
    # Capture stderr to suppress warnings
    f = StringIO()
    with redirect_stderr(f):
        # Open file dialog for multiple image files
        selected_images = filedialog.askopenfilenames(
            title="Select Discharge Summary Images",
            initialdir=initial_dir,
            filetypes=[
                ("Image files", "*.jpg *.jpeg *.png *.gif *.bmp *.tiff"),
                ("JPEG files", "*.jpg *.jpeg"),
                ("PNG files", "*.png"),
                ("All files", "*.*")
            ]
        )
    
    # Clean up the tkinter root window
    root.destroy()
    
    # Convert to list and make paths relative to current directory
    selected_images = list(selected_images)
    if selected_images:
        # Convert absolute paths to relative paths if they're in the images directory
        relative_images = []
        current_dir = os.getcwd()
        for img_path in selected_images:
            try:
                rel_path = os.path.relpath(img_path, current_dir)
                relative_images.append(rel_path)
            except ValueError:
                # If relpath fails, use absolute path
                relative_images.append(img_path)
        
        selected_images = relative_images
        
        print(f"\nSelected {len(selected_images)} files:")
        for file in selected_images:
            print(f"  - {file}")
        print("\nYou can now run the OCR cell with these selected files.")
    else:
        print("\nNo files selected.")
        selected_images = []

except ImportError:
    print("tkinter not available. Using fallback file selection...")
    # Fallback to showing available files
    image_files = glob.glob("images/*.jpeg") + glob.glob("images/*.jpg") + glob.glob("images/*.png")
    image_files = [f for f in image_files if not f.endswith('.DS_Store')]
    image_files.sort()
    
    print("Available image files:")
    for i, file in enumerate(image_files, 1):
        print(f"{i}. {file}")
    
    # Use all available files as default
    selected_images = image_files
    print(f"\nUsing all {len(selected_images)} available files:")
    for file in selected_images:
        print(f"  - {file}")

Opening file selection dialog...
You can select multiple files by holding Cmd (Mac) or Ctrl (Windows/Linux)


2025-08-26 13:35:51.979 Python[39556:3283086] The class 'NSOpenPanel' overrides the method identifier.  This method is implemented by class 'NSWindow'



Selected 4 files:
  - images/3.4.jpeg
  - images/3.1.jpeg
  - images/3.2.jpeg
  - images/3.3.jpeg

You can now run the OCR cell with these selected files.


In [ ]:
## OCR ##
# Use the selected images from the file selection above
if 'selected_images' in globals() and selected_images:
    images = selected_images
    print(f"Processing {len(images)} selected files:")
    for img in images:
        print(f"  - {img}")
else:
    # Fallback to default files if none selected
    images = ["images/3.1.jpeg", "images/3.2.jpeg", "images/3.3.jpeg", "images/3.4.jpeg"]
    print("No files selected, using default files:")
    for img in images:
        print(f"  - {img}")

print("\nRunning OCR...")
markdown = app_graph.run_node("OCR", images, "gpt-4o")

# Clean up markdown text - remove code block wrappers if present
if markdown.startswith("```markdown\n"):
    markdown = markdown[12:]  # Remove "```markdown\n"
if markdown.endswith("\n```"):
    markdown = markdown[:-4]  # Remove "\n```"
elif markdown.endswith("```"):
    markdown = markdown[:-3]  # Remove "```"

# Also handle case where it starts with just ```
if markdown.startswith("```\n"):
    markdown = markdown[4:]  # Remove "```\n"

# Save the cleaned markdown to a file
with open("discharge.md", "w", encoding="utf-8") as f:
    f.write(markdown)
print("Markdown saved to discharge.md")

# Display the markdown in the notebook
display(Markdown(markdown))

Processing 4 selected files:
  - images/3.4.jpeg
  - images/3.1.jpeg
  - images/3.2.jpeg
  - images/3.3.jpeg

Running OCR...
=== OCR Node Output (Model: gpt-4o) ===
```markdown
# SHUSRUSHA
A UNIT OF SHUSRUSHA NURSING HOME PVT. LTD.
P-290, C.I.T. Scheme VIM, Swami Swarupananda Sarani, Kolkata - 700 054  
Phone : 2362 8863, 2362 8430, 2364 8910

## DISCHARGE / TRANSFER / DISCHARGE AGAINST MEDICAL ADVISE CERTIFICATE

**Name:** Mrs. Bina Jaicwal  
**UHID No.:** 191020039341  
**Bed No.:** 109  
**Sex:** Female  
**Age:** 64 yrs  
**Address:** 40 Mr. Rohit Jaicwal, Raj, Bholanath Sarani Road, Teghoria, Rajarhat, Gopalpur, Baguihati 700157  

**Date of Admission:** 24/8/15  
**Time of Admission:** 10:00 AM  
**Date of Discharge:** 26/8/15  
**Time of Discharge:**  

I, Dr. Sunil Tibrewal hereby certify that the above-mentioned patient was under my treatment in this Nursing Home.

### REASONS FOR ADMISSION : PRESENTING COMPLAINTS
- Vomiting
- Abdominal distension
- Swelling

### FINAL DIAGNOS

In [ ]:
## EXTRACT DIAGNOSES ##
diagnoses = app_graph.run_node("ExtractDiagnoses", model="gpt-4o-mini")

In [ ]:
## EXTRACT MEDICATIONS ##
medications = app_graph.run_node("ExtractMedications", model="gpt-4o-mini")


In [ ]:
## FIX MEDICATIONS ##
fixed_medications = app_graph.run_node("FixMedications", model="gpt-4o")

In [ ]:
## GENERATE SUMMARY ##
html_summary = app_graph.run_node("AddSummaryPills")

# Save the HTML summary to a file
with open("summary.html", "w", encoding="utf-8") as f:
    f.write(html_summary)
print("Summary saved to summary.html")

# Display the HTML in the notebook
display(HTML(html_summary))